In [10]:
#!pip install emoji nltk torch transformers

In [11]:
import pandas as pd
import re
import emoji
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to
[nltk_data]     /Users/rishushrivastava/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [12]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/hate-speech/labeled_data.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/data/hate-speech/labeled_data.csv'

In [13]:
data.describe().T

NameError: name 'data' is not defined

In [5]:
data.groupby(['class']).count()['tweet']

class
0     1430
1    19190
2     4163
Name: tweet, dtype: int64

In [6]:
class_label = {0:'hate_speech',1:'offensive_language',2:'neither'}

In [7]:
data_refactor = data.copy()

data_refactor['label'] = data_refactor['class'].map(class_label)
data_refactor.drop(['Unnamed: 0','count','hate_speech','offensive_language','neither'],inplace=True, axis=1)
data_refactor.rename(columns={'class':'category'}, inplace=True)

data = data_refactor.copy()
data.head()

,category,tweet,label
0,2,!!! RT @mayasolovely: As a woman you shouldn't...,neither
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,offensive_language
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,offensive_language
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,offensive_language
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,offensive_language


In [8]:
data[data['category'] == 0]['tweet'][:3]

85     "@Blackman38Tide: @WhaleLookyHere @HowdyDowdy1...
89     "@CB_Baby24: @white_thunduh alsarabsss" hes a ...
110    "@DevilGrimz: @VigxRArts you're fucking gay, b...
Name: tweet, dtype: object

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_val, y_train, y_val = train_test_split(
    data.index.values,
    data.category.values,
    test_size=0.15,
    random_state=17,
    stratify=data.category.values,    
)

y_train

array([0, 1, 0, ..., 1, 2, 1])

In [11]:
data['data_type'] = ['not_set']*data.shape[0]
data.loc[X_train, 'data_type'] = 'train'
data.loc[X_val, 'data_type'] = 'val'

data.head()

,category,tweet,label,data_type
0,2,!!! RT @mayasolovely: As a woman you shouldn't...,neither,train
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,offensive_language,train
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,offensive_language,train
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,offensive_language,val
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,offensive_language,train


In [12]:
data.groupby(['category', 'label', 'data_type']).count()

tweet
category label              data_type       
0        hate_speech        train       1216
                            val          214
1        offensive_language train      16311
                            val         2879
2        neither            train       3538
                            val          625

In [2]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
import torch

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
print(tokenizer)


PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [15]:
encoded_data_train = tokenizer.batch_encode_plus(
    data[data.data_type=='train'].tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)

In [16]:
encoded_data_val = tokenizer.batch_encode_plus(
    data[data.data_type=='val'].tweet.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)

In [17]:
print(encoded_data_train['input_ids'][0])
print(encoded_data_train['attention_mask'][0])

tensor([  101,   999,   999,   999, 19387,  1030,  9815, 19454, 21818,  2135,
         1024,  2004,  1037,  2450,  2017,  5807,  1005,  1056, 17612,  2055,
         9344,  2039,  2115,  2160,  1012,  1004, 23713,  1025,  2004,  1037,
         2158,  2017,  2323,  2467,  2202,  1996, 11669,  2041,  1012,  1012,
         1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [18]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data[data.data_type=='train'].category.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(data[data.data_type=='val'].category.values)

In [19]:
dataset_train = TensorDataset(
    torch.LongTensor(input_ids_train),
    torch.LongTensor(attention_masks_train),
    labels_train
)

dataset_val = TensorDataset(
    torch.LongTensor(input_ids_val),
    torch.LongTensor(attention_masks_val),
    labels_val
)

In [6]:
from transformers import BertForSequenceClassification

In [7]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels = len(class_label),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

NameError: name 'class_label' is not defined

In [5]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [23]:
#batch_size = 32

dataloader_train = DataLoader(dataset_train,sampler=RandomSampler(dataset_train),batch_size=4)
dataloader_val = DataLoader(dataset_val,sampler=RandomSampler(dataset_val),batch_size=2)

In [4]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [25]:
optimizer = AdamW(model.parameters(),lr=1e-5,eps=1e-8)
epochs = 5
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

device == 'cpu'

NameError: name 'model' is not defined

In [27]:
import random
from tqdm.notebook import tqdm

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [28]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for v, k in class_label.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [29]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [1]:
import datetime

today = datetime.datetime.now().strftime('%Y%m%d%H%M%S')

model_name = "/content/drive/MyDrive/Colab Notebooks/model/hate-speech/hatespeech-finetuned-bert-{}".format(today)

for epoch in tqdm(range(epochs)):
    
    model.train()
    
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }
        
        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    
    torch.save(model.state_dict(), model_name)

NameError: ignored

In [8]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(class_label),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

NameError: name 'class_label' is not defined

In [ ]:
model_name = "model/hatespeech-finetuned-bert-20210724120229"

model.load_state_dict(torch.load(model_name, map_location=torch.device('cpu')))

In [ ]:
accuracy_per_class(predictions, true_vals)